In [34]:
# OpenAIの作成

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager

llama_debug_handler = LlamaDebugHandler(print_trace_on_end=False)
callback_manager = CallbackManager([llama_debug_handler])

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512
Settings.callback_manager = callback_manager

In [35]:
# Nebula Graphのコンフィグ

import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ["NEBULA_ADDRESS"] = (
    "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally
)

space_name = "boxerengine"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [36]:
# NebulaGraphの接続
# 「llamaindexのスペースの作成手順」を実施後に実行する

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [37]:
# Documentのサンプル情報を取得する

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["水平対向エンジン"], auto_suggest=False, lang_prefix="ja"
)

In [38]:
# KnowledgeGraphIndexを構築する

from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    # space_name=space_name,
    # edge_types=edge_types,
    # rel_prop_names=rel_prop_names,
    # tags=tags,
    include_embeddings=True,
)

In [39]:
# Nebula Graphに接続してSpaceの情報を取得する
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password root

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created


,Name
0,boxerengine
1,llamaindex


In [41]:
# llamaindexの検索を行う

%ngql USE boxerengine;
%ngql MATCH ()-[e]->() RETURN e LIMIT 20

,e
0,"(""12気筒"")-[:relationship@2847039626096127544{relationship: ""一次偶力を相殺できる""}]->(""180度v型"")"
1,"(""12気筒"")-[:relationship@2847039626096127544{relationship: ""一次偶力を相殺できる""}]->(""180度v型でも"")"
2,"(""12気筒エンジン"")-[:relationship@-4259500649969518903{relationship: ""圧倒的に""}]->(""180度v型"")"
3,"(""12気筒エンジン"")-[:relationship@-4259500649969518903{relationship: ""圧倒的に""}]->(""180度v型が多い"")"
4,"(""12気筒エンジン"")-[:relationship@-2183616879379552818{relationship: ""全長を""}]->(""短縮する"")"
5,"(""180度v型"")-[:relationship@-7182355429830258513{relationship: ""60°の等間隔燃焼を""}]->(""得られる"")"
6,"(""180度v型"")-[:relationship@-4235052806840745764{relationship: ""使用できる""}]->(""フラットプレーンクランクシャフト"")"
7,"(""180度v型"")-[:relationship@-2648541590415519139{relationship: ""位相をずらすためのクランクウエブが不要で""}]->(""クランクシャフトの全長と剛性でボクサー構成よりも有利"")"
8,"(""180度v型"")-[:relationship@-2252351635012999292{relationship: ""一次振動・二次振動・偶力振動とも""}]->(""釣り合いが取れる"")"
9,"(""180度v型"")-[:relationship@-2195781045337484359{relationship: ""レイアウトは""}]->(""レース用エンジンに適している"")"


In [24]:
# draw the result
# 結果が出力されない
%ng_draw

<class 'pyvis.network.Network'> |N|=121 |E|=100

In [64]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever,
)

/tmp/ipykernel_71274/833418821.py:4: DeprecationWarning: Call to deprecated class KnowledgeGraphRAGRetriever. (KnowledgeGraphRAGRetriever is deprecated, it is recommended to use PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  graph_rag_retriever = KnowledgeGraphRAGRetriever(


In [65]:
from IPython.display import display, Markdown

llama_debug_handler.flush_event_logs()
response = query_engine.query(
    "ボクサーエンジンが採用される理由を教えて下さい",
)
display(Markdown(f"{response}"))

ボクサーエンジンが採用される理由は、全高を低くすることができ、左右に水平に配置されるため、低重心を実現できる点が挙げられます。

In [66]:
def print_llm_logs(logs):
    for event in logs:
        print("\033[31m~~~~~~~~ Event ~~~~~~~~\033[0m")
        _, response = (event[0], event[1])
        messages = response.payload["messages"]
        response = response.payload["response"]
        print("\033[44m~~~~ Request ~~~~\033[0m")
        for message in messages:
            print(f"\033[32mRole: {message.role}\033[0m")
            print(f"content: {message.content}")
        print("\033[46m~~~~~ Response ~~~~\033[0m")
        print(f"Role: {response.message.role}")
        print(f"content: {response.message.content}")


print_llm_logs(llama_debug_handler.get_llm_inputs_outputs())

~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
ボクサーエンジンが採用される理由を教えて下さい
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: KEYWORDS: ボクサーエンジン, 採用, 理由
~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: 
Generate synonyms or possible form of keywords up to 5 in total,
considering possible cases of capitalization, pluralization, common expressions, etc.
Provide all synonyms of keywords in comma-separated format: 'SYNONYMS: <keywords>'
Note, result should be in one-line with only one 'SYNONYMS: ' prefix
----
KEYWORDS: ['理由', 'ボクサーエンジン', '採用']
----

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: SYNONYMS: 理由, ボ

In [67]:
from IPython.display import display, Markdown

llama_debug_handler.flush_event_logs()
response = query_engine.query(
    "水平対向エンジンが使用されている車種を全て教えて下さい。",
)
display(Markdown(f"{response}"))

Porsche、Subaru、トヨタ

In [68]:
print_llm_logs(llama_debug_handler.get_llm_inputs_outputs())

~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
水平対向エンジンが使用されている車種を全て教えて下さい。
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: KEYWORDS: 水平対向エンジン, 車種, 使用
~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: 
Generate synonyms or possible form of keywords up to 5 in total,
considering possible cases of capitalization, pluralization, common expressions, etc.
Provide all synonyms of keywords in comma-separated format: 'SYNONYMS: <keywords>'
Note, result should be in one-line with only one 'SYNONYMS: ' prefix
----
KEYWORDS: ['水平対向エンジン', '使用', '車種']
----

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: SYNONYMS: 

In [69]:
from IPython.display import display, Markdown

llama_debug_handler.flush_event_logs()
response = query_engine.query(
    "水平対向エンジンが使用されている車の名前を全て教えて下さい。",
)
display(Markdown(f"{response}"))

Subaru

In [70]:
print_llm_logs(llama_debug_handler.get_llm_inputs_outputs())

~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
水平対向エンジンが使用されている車の名前を全て教えて下さい。
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: KEYWORDS: 水平対向エンジン, 車, 名前, 使用
~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: 
Generate synonyms or possible form of keywords up to 5 in total,
considering possible cases of capitalization, pluralization, common expressions, etc.
Provide all synonyms of keywords in comma-separated format: 'SYNONYMS: <keywords>'
Note, result should be in one-line with only one 'SYNONYMS: ' prefix
----
KEYWORDS: ['水平対向エンジン', '使用', '名前', '車']
----

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: 

In [71]:
from IPython.display import display, Markdown

llama_debug_handler.flush_event_logs()
response = query_engine.query(
    "水平対向エンジンの歴史を教えて下さい。",
)
display(Markdown(f"{response}"))

Subaruは1972年から水平対向エンジンを採用しており、そのエンジンはスバル・レガシィやスバル・レオーネ、スバル・インプレッサなどの車種に搭載されています。水平対向エンジンは排気系の下側に配置され、太い配管や排ガス対策に影響を与えています。他にも水平対向エンジンはボクサーエンジンとしても知られ、様々な自動車メーカーによって開発や生産が行われています。

In [72]:
print_llm_logs(llama_debug_handler.get_llm_inputs_outputs())

~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
水平対向エンジンの歴史を教えて下さい。
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: KEYWORDS: 水平対向エンジン, 歴史, 教える
~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: 
Generate synonyms or possible form of keywords up to 5 in total,
considering possible cases of capitalization, pluralization, common expressions, etc.
Provide all synonyms of keywords in comma-separated format: 'SYNONYMS: <keywords>'
Note, result should be in one-line with only one 'SYNONYMS: ' prefix
----
KEYWORDS: ['水平対向エンジン', '歴史', '教える']
----

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: SYNONYMS: 水平対向エンジ

In [73]:
from IPython.display import display, Markdown

llama_debug_handler.flush_event_logs()
response = query_engine.query(
    "水平対向エンジンの歴史を搭載車両を含めて教えて下さい。",
)
display(Markdown(f"{response}"))

Subaruは水平対向エンジンを搭載している車両の歴史を持っており、1972年から水平対向エンジンで駆動する前置きエンジン・4輪駆動配置やスバル・レガシィ、スバル・レオーネ、スバル・インプレッサなどの車両に水平対向エンジンが搭載されています。

In [74]:
print_llm_logs(llama_debug_handler.get_llm_inputs_outputs())

~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: A question is provided below. Given the question, extract up to 5 keywords from the text. Focus on extracting the keywords that we can use to best lookup answers to the question. Avoid stopwords.
---------------------
水平対向エンジンの歴史を搭載車両を含めて教えて下さい。
---------------------
Provide keywords in the following comma-separated format: 'KEYWORDS: <keywords>'

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: KEYWORDS: 水平対向エンジン, 歴史, 搭載車両
~~~~~~~~ Event ~~~~~~~~
~~~~ Request ~~~~
Role: MessageRole.USER
content: 
Generate synonyms or possible form of keywords up to 5 in total,
considering possible cases of capitalization, pluralization, common expressions, etc.
Provide all synonyms of keywords in comma-separated format: 'SYNONYMS: <keywords>'
Note, result should be in one-line with only one 'SYNONYMS: ' prefix
----
KEYWORDS: ['水平対向エンジン', '歴史', '搭載車両']
----

~~~~~ Response ~~~~
Role: MessageRole.ASSISTANT
content: SYNONYM